In this tutorial, you will use the Keras Turner to perform hypertuning for an image classification application.

## Setup

In [1]:
import tensorflow as tf
from tensorflow import keras

In [3]:
!pip install -q -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 kB 7.4 MB/s eta 0:00:00


In [4]:
import keras_tuner as kt

## Download and prepare the dataset

In this tutorial, you will use the Keras Tuner to find the best hyperparameters for the Fashion MNIST dataset.

In [5]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [6]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

## Define the model

When you build a model for hypertuning, you also define the hyperparameter search space in addition to the model architecture. The model you set up for hypertuning is called a hypermodel.

You can define a hypermodel through two appreacher:
* By using a model builder function
* By subclassing the `HyperModel` class of the Keras Tuner API

In this tutorial, you use a model builder function to define the image classification model. The model builder function returns a compiled model and uses hyperparameters you define inline to hypertune the model.

In [18]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation="relu"))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=["accuracy"])
  
  return model

## Instantiate the tuner and perform hypertuning

Instantiate the tuner tto perform the hypertuning. The Keras Tuner has four tuners available - `RandomSearch`, `Hyperband`, `BayesianOptimzation`, and `Sklearn`. In this tutorial, you use the `Hyperband` tuner.

To instantiate the Hyperband tuener, you must specify the hypermodel, the `objective` to optimize and the maximum number of epochs to train (`max_epochs`).

In [19]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [20]:
stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [21]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 26 Complete [00h 01m 05s]
val_accuracy: 0.8744166493415833

Best val_accuracy So Far: 0.8933333158493042
Total elapsed time: 00h 16m 00s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 384 and the optimal learning rate for the optimizer
is 0.001.



In [22]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 10s 6ms/step - loss: 0.4954 - accuracy: 0.8251 - val_loss: 0.4062 - val_accuracy: 0.8541
Epoch 2/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3690 - accuracy: 0.8658 - val_loss: 0.3630 - val_accuracy: 0.8684
Epoch 3/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3298 - accuracy: 0.8800 - val_loss: 0.3615 - val_accuracy: 0.8712
Epoch 4/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3056 - accuracy: 0.8871 - val_loss: 0.3475 - val_accuracy: 0.8730
Epoch 5/50
1500/1500 [==============================] - 8s 6ms/step - loss: 0.2875 - accuracy: 0.8940 - val_loss: 0.3425 - val_accuracy: 0.8767
Epoch 6/50
1500/1500 [==============================] - 10s 6ms/step - loss: 0.2712 - accuracy: 0.8991 - val_loss: 0.3248 - val_accuracy: 0.8799
Epoch 7/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2588 - accuracy: 0.9033 - val_loss: 0.3090 - val_accurac

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/39
1500/1500 [==============================] - 12s 7ms/step - loss: 0.4990 - accuracy: 0.8254 - val_loss: 0.4253 - val_accuracy: 0.8453
Epoch 2/39
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3737 - accuracy: 0.8644 - val_loss: 0.4096 - val_accuracy: 0.8518
Epoch 3/39
1500/1500 [==============================] - 10s 7ms/step - loss: 0.3317 - accuracy: 0.8780 - val_loss: 0.3399 - val_accuracy: 0.8777
Epoch 4/39
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3056 - accuracy: 0.8872 - val_loss: 0.3384 - val_accuracy: 0.8772
Epoch 5/39
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2886 - accuracy: 0.8921 - val_loss: 0.3373 - val_accuracy: 0.8816
Epoch 6/39
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2733 - accuracy: 0.8990 - val_loss: 0.3178 - val_accuracy: 0.8852
Epoch 7/39
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2586 - accuracy: 0.9032 - val_loss: 0.3275 - val_accurac

In [ ]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 2ms/step - loss: 0.4940 - accuracy: 0.8896
[test loss, test accuracy]: [0.49400606751441956, 0.8895999789237976]
